In [1]:
import numpy as np
from qiskit import *
from qiskit.providers.jobstatus import JobStatus
#Call provider and set token value
from qiskit_textbook.tools import array_to_latex

from signature import *
from hash import crypto_hash
from swap_test import swap_test

We generate a legitimate signature and a forged signature. In the forged signature, the private keys do do not correspond to the public keys. We attempt to validate each signature.

Let's say Alice wants to trasfer 1 qrypto BTC to Bob....

In [2]:
alices_signature = Signed_Transaction("A-->B(1qBTC)").get_signature()

In [3]:
print("num of bits in transaction: ", len(alices_signature))
print()
print("first bit signature:")
print("bit: ", alices_signature[0]['m_bit'])
print("Private keys: ", alices_signature[0]['priv_keys'])

num of bits in transaction:  78

first bit signature:
bit:  1
Private keys:  ['1110', '1001', '1000']


In [4]:
print("Some Public keys for the first bit: ")
print()

for i in alices_signature[0]['pub_keys']['f_k0']:
    print(array_to_latex(i))

Some Public keys for the first bit: 



<IPython.core.display.Math object>

None


<IPython.core.display.Math object>

None


<IPython.core.display.Math object>

None


Where did these public come from?? A Quantum One-way Hash Function! We display a quantum circuit used for hashing. The private key to be hashed is the integer 9:

In [5]:
state = crypto_hash('1001', ['1011', '1101', '1001', '0010'], verbose=True)

         ┌───┐                                                            »
q1810_0: ┤ H ├──────o────────────■───────────o───────────■──────────o─────»
         ├───┤      │            │           │           │          │     »
q1810_1: ┤ H ├──────o────────────o───────────■───────────■──────────o─────»
         └───┘┌─────┴─────┐┌─────┴─────┐┌────┴─────┐┌────┴────┐┌────┴────┐»
q1810_2: ─────┤ RY(11π/4) ├┤ RY(13π/4) ├┤ RY(9π/4) ├┤ RY(π/2) ├┤ RY(22π) ├»
              └───────────┘└───────────┘└──────────┘└─────────┘└─────────┘»
«                                         
«q1810_0: ─────■──────────o─────────■─────
«              │          │         │     
«q1810_1: ─────o──────────■─────────■─────
«         ┌────┴────┐┌────┴────┐┌───┴────┐
«q1810_2: ┤ RY(26π) ├┤ RY(18π) ├┤ RY(4π) ├
«         └─────────┘└─────────┘└────────┘


The state vector for the resulting quantum state:

In [6]:
array_to_latex(state, pretext="\\text{Hash('1001')} = ")

<IPython.core.display.Math object>

Now let's validate Alice's signature:

In [7]:
alice_validation = Validation(alices_signature)
print(alice_validation.validate())

{'pass': True, 'bits_failure': '0.0%'}


Alices message passed! Now let's say Eve wants to forge a transaction, say using Bob's public keys...

In [8]:
eves_transaction = Forgery("B-->E(1qBTC)").get_signature()
eve_validation = Validation(eves_transaction)
print(eve_validation.validate())

{'pass': False, 'bits_failure': '75.641%'}


We see Eve's Signature failed! 81% of the bits did not pass. How was this done? A Swap test! We show a the circuit for a swap test. The swap test tries to determine whether two quantum states are equal:

In [12]:
pub_0 = alices_signature[0]['pub_keys']['f_k0'][0]
pub_1 = alices_signature[0]['pub_keys']['f_k0'][0]

swap_test(pub_0, pub_1, verbose=True)

                        ┌───┐                                         ┌───┐┌─┐
q8259_0: ───────────────┤ H ├────────────────────────────■─────────■──┤ H ├┤M├
         ┌──────────────┴───┴───────────────┐┌───┐       │  ┌───┐  │  └───┘└╥┘
q8259_1: ┤0                                 ├┤ X ├───────■──┤ X ├──┼────────╫─
         │  initialize(0.70711,0.70711,0,0) │└─┬─┘┌───┐  │  └─┬─┘  │  ┌───┐ ║ 
q8259_2: ┤1                                 ├──┼──┤ X ├──┼────┼────■──┤ X ├─╫─
         ├──────────────────────────────────┤  │  └─┬─┘┌─┴─┐  │    │  └─┬─┘ ║ 
q8259_3: ┤0                                 ├──■────┼──┤ X ├──■────┼────┼───╫─
         │  initialize(0.70711,0.70711,0,0) │       │  └───┘     ┌─┴─┐  │   ║ 
q8259_4: ┤1                                 ├───────■────────────┤ X ├──■───╫─
         └──────────────────────────────────┘                    └───┘      ║ 
 c470: 1/═══════════════════════════════════════════════════════════════════╩═
                                                    

0

Swap test outputs 0, Success!